In [ ]:
import keras
from keras import backend as K
from keras.models import Sequential, Model
from keras.layers import Input, LSTM, RepeatVector, CuDNNLSTM
from keras.layers.core import Flatten, Dense, Dropout, Lambda
from keras.optimizers import SGD, RMSprop, Adam
from keras import objectives
from keras.callbacks import ReduceLROnPlateau

import numpy as np
import matplotlib.pyplot as plt
import os, math

In [ ]:
data = np.load('absolute_interpolated_data.npy')[:,:,:2] #Data generated by preprocessing script

# Check if data contains nan
if np.isnan(data).any():
    print("Data contains nan values")

print("size: ",data.nbytes/1e6, 'MB')
print('shape:', data.shape)

# Define Model

In [ ]:
#Some hyperparameters

n_datapoints, timesteps, input_dim = data.shape

batch_size  = 500
intermediate_dim  = 512
latent_dim = 64
epsilon_std= 1
beat = 1.25

In [ ]:
def sample_z(args):
    z_mean, z_log_sigma = args
    epsilon = K.random_normal(shape=(batch_size, latent_dim),
                              mean=0., stddev=epsilon_std)
    return z_mean + K.exp(z_log_sigma) * epsilon

In [ ]:
vae_input = Input(shape=(timesteps, input_dim,))

# LSTM encoding
encoder_layer = CuDNNLSTM(intermediate_dim, return_sequences=True)(vae_input)
encoder_layer = CuDNNLSTM(intermediate_dim)(encoder_layer)

# VAE Z layer
z_mean = Dense(latent_dim)(encoder_layer)
z_log_sigma = Dense(latent_dim)(encoder_layer)

z = Lambda(sample_z, output_shape=(latent_dim,))([z_mean, z_log_sigma])

# decoded LSTM layer
decoder_layer_1 = CuDNNLSTM(intermediate_dim, return_sequences=True)
decoder_layer_2 = CuDNNLSTM(intermediate_dim, return_sequences=True)

decoder_output = LSTM(input_dim, return_sequences=True, activation = None)


decoded_layer = RepeatVector(timesteps)(z)
decoded_layer = decoder_layer_1(decoded_layer)
decoded_layer = decoder_layer_2(decoded_layer)


# decoded layer
vae_output = decoder_output(decoded_layer)

# end-to-end autoencoder
vae = Model(vae_input, vae_output)

In [ ]:
# encoder, from inputs to latent space
encoder_output_mean = z_mean
encoder_output_std = Lambda(K.exp)(z_log_sigma)
encoder = Model(vae_input, [encoder_output_mean, encoder_output_std])

In [ ]:
# decoder, from latent space to reconstructed inputs
decoder_input = Input(shape=(latent_dim,))

_decoded_layer = RepeatVector(timesteps)(decoder_input)
_decoded_layer = decoder_layer_1(_h_decoded)
_decoded_layer = decoder_layer_2(_h_decoded)

decoder_output = decoder_output(_h_decoded)

generator = Model(decoder_input, decoder_output)

In [ ]:
def loss_function(x, x_decoded_mean):
    cross_entropy = objectives.mse(x, x_decoded_mean)
    kl_loss = - 0.5 * K.mean(1 + z_log_sigma - K.square(z_mean) - K.exp(z_log_sigma))
    loss = cross_entropy + beta*kl_loss
    return loss

vae.compile(optimizer='rmsprop', loss=loss_function)
vae.summary()

In [ ]:
cb = [ReduceLROnPlateau(patience = 5)]

history = vae.fit(x = data, 
                  y = data,
                  validation_split= 0.1,
                  batch_size=batch_size, 
                  epochs = 100,
                  callbacks = cb) 

In [ ]:
vae.save_weights('vae_weights.hdf5')
decoder.save_weights('decoder_weights.hdf5')
encoder.save_weights('encoder_weights.hdf5')
np.save('history_final_training.npy',history.history)
np.save('final_preds_train.npy', vae_output)
np.save('final_labels_train.npy', labels[:batch_size])
np.save('final_preds.npy', vae_output)
np.save('final_labels.npy', labels[:batch_size])

# Plot Sequence

In [ ]:
d = 50
dp = vae_output[d]

fig, ax = plt.subplots(1,2)
x_output = dp[:,0]
y_output = dp[:,1]
x_input = p_data[d,:,0]
y_input = p_data[d,:,1]



i = 0
for x, y in zip(x_input[::2], y_input[::2]):
    ax[0].text(x+0.1, y+0.1, str(i), color="black", fontsize=12)
    i+=1
    
ax[0].plot(x_input, y_input,'ko', markersize = 4)
ax[0].axis('equal')
ax[0].axis('off')

i = 0
for x, y in zip(x_output[::2], y_output[::2]):
    ax[1].text(x+0.1, y+0.1, str(i), color="black", fontsize=12)
    i+=1

ax[1].plot(x_output, y_output,'bo', markersize = 4)
ax[1].axis('equal')
ax[1].axis('off')
plt.show()

# Plot multiple decodings

In [ ]:
p_data = data[-batch_size:,:,:]
vae_output = vae.predict(data[-batch_size:,:,:], batch_size = batch_size)
# Read the labels from trainlabels.txt and testlabels.txt in the sequences directory
dataset_dir = os.path.abspath('sequences') # Name of your sequences folder
labels_file = os.path.join(dataset_dir, 'trainlabels.txt')
f = open(labels_file)
train_labels = np.array([int(x) for x in f.read().split()])
f.close()
labels_file = os.path.join(dataset_dir, 'testlabels.txt')
f = open(labels_file)
test_labels = np.array([int(x) for x in f.read().split()])
labels = np.concatenate((train_labels, test_labels))

# Input to this code section
# variables
# `labels`     - the correct labels of the data in variable `data`
# `data`       - the preprocessed data
# `vae_output` - the outputs generated by the VAE when feeding `data` to the network
# `dp_inds`    - select a subset of indices to plot (type: list or tuple)

#vae_output = data
dp_inds = range(20,28)

axis_max = 3
axis_min = -3
n_rows = math.ceil(len(dp_inds) / 2)
fig, axs = plt.subplots(n_rows, 4, figsize=(20, 5*n_rows))
for i, di in enumerate(dp_inds):
    ax_row = i // 2
    ax_col = 0 if i % 2 == 0 else 2
   # plot_indices = []
   # for j in range(data.shape[1]):
   #     plot_indices.append(np.any(np.abs(data[di,j,[0,1]]) > 1e-4))
    
    axs[ax_row, ax_col].plot(p_data[di,:,0], p_data[di,:,1],'ko', markersize = 5)
    axs[ax_row, ax_col].axis('equal')
    axs[ax_row, ax_col].set_xlim(axis_min, axis_max)
    axs[ax_row, ax_col].set_ylim(axis_min, axis_max)
    axs[ax_row, ax_col].grid(True)
    axs[ax_row, ax_col].set_title("Input to VAE digit {}".format(labels[di]))
    axs[ax_row, ax_col].axis('off')
    axs[ax_row, ax_col + 1].plot(vae_output[di,:,0], vae_output[di,:,1],'bo', markersize = 5)
    axs[ax_row, ax_col + 1].axis('equal')
    axs[ax_row, ax_col + 1].set_xlim(axis_min, axis_max)
    axs[ax_row, ax_col + 1].set_ylim(axis_min, axis_max)
    axs[ax_row, ax_col + 1].grid(True)
    axs[ax_row, ax_col + 1].set_title("VAE generated digit {}".format(labels[di]))
    axs[ax_row, ax_col + 1].axis('off')

plt.show()

# Create Morph

In [ ]:
d1 = 0
d2 = 15

h1 = encoder.predict(data[d1:d1+1])
z1 = h1[0] + h1[1]*np.random.randn(64)
h2 = encoder.predict(data[d2:d2+1])
z2 = h2[0] + h2[1]*np.random.randn(64)


n = 10
alpha = np.linspace(0,1, num=n)
target = np.zeros((n,64))

for i in range(n):
    target[i,:] = (1-alpha[i])*z1 + alpha[i]*z2

outputs = np.zeros((n,60,2))
for i in range(n):
    seq = decoder.predict(target[i:i+1,:])
    outputs[i,:,:] = seq
    
fig, axes = plt.subplots(1,n, figsize = (15,5))

from matplotlib.pyplot import show
for i in range(n):
    ax = axes[i]
    ax.plot(outputs[i,:,0], outputs[i,:,1],'ko')
    ax.axis('equal')
    ax.axis('off')
show()

# Sample multiple ouputs

In [ ]:
d1 = 21

h1 = encoder.predict(data[d1:d1+1])

fig, axes = plt.subplots(1,5)

for i in range(5):
    z1 = h1[0] + h1[1]*2*np.random.randn(64)
    seq = decoder.predict(z1)
    
    x_output = seq[0,:,0]
    y_output = seq[0,:,1]
    j = 0
    
    axes[i].plot(x_output, y_output,'ko', markersize = 4)
    axes[i].axis('equal')
    axes[i].axis('off')